Import Libraries

In [ ]:

import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO, RTDETR
import multiprocessing
import warnings
warnings.filterwarnings("ignore")
import random
from datetime import datetime
import time
from glob import glob
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from PIL import Image
import torch
import numpy as np
import pprint



Configurations

In [2]:
class CFG:
    seed = 42
    random_state = 42
    folds=5
    device = 'cuda' if torch.cuda.is_available() else "cpu"

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)
seed_everything(CFG.seed)

Set Paths and Load CSV

In [3]:
# Set the data directory
ROOT_DIR  = Path(os.getcwd())
IMGS_DIR = ROOT_DIR / 'images'

# load csvs
train = pd.read_csv(ROOT_DIR / 'non_duplicates_train.csv') 
test = pd.read_csv(ROOT_DIR / 'Test.csv')
ss = pd.read_csv(ROOT_DIR / 'SampleSubmission.csv')
train.head()

,Unnamed: 0,Image_ID,confidence,class,ymin,xmin,ymax,xmax
0,0,id_11543h.jpg,1,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928
1,1,id_11543h.jpg,1,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724
2,2,id_11543h.jpg,1,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645
3,3,id_11543h.jpg,1,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316
4,4,id_11ee1c.jpg,1,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952


Pre-processing

In [4]:
# assign unique integer to each crop disease
unique_classes = train['class'].unique()
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
reverse_class_mapping = {class_mapping[key]:key for key in class_mapping}  # reverse class mapping
pprint.pp(f'class_mapping_dict: {class_mapping}')

("class_mapping_dict: {'Pepper_Bacterial_Spot': 0, 'Pepper_Fusarium': 1, "
 "'Corn_Cercospora_Leaf_Spot': 2, 'Corn_Common_Rust': 3, 'Pepper_Leaf_Curl': "
 "4, 'Tomato_Early_Blight': 5, 'Pepper_Cercospora': 6, 'Tomato_Septoria': 7, "
 "'Pepper_Leaf_Mosaic': 8, 'Corn_Streak': 9, 'Corn_Healthy': 10, "
 "'Pepper_Healthy': 11, 'Tomato_Healthy': 12, 'Tomato_Late_Blight': 13, "
 "'Tomato_Fusarium': 14, 'Pepper_Leaf_Blight': 15, 'Tomato_Leaf_Curl': 16, "
 "'Tomato_Bacterial_Spot': 17, 'Tomato_Mosaic': 18, "
 "'Corn_Northern_Leaf_Blight': 19}")


In [43]:
# remove duplicate detections (boxes) if any
defined_cols = ['Image_ID',	'confidence',	'class',	'ymin',	'xmin',	'ymax',	'xmax']
train = train[defined_cols]
print(f'Sum of duplicated colums: {train.duplicated().sum()}')
print(f'Size of dataframe before removing duplicates: {train.shape}')
# remove duplicates
train = train.drop_duplicates()
print(f'Sum of duplicated colums after removing duplicates: {train.duplicated().sum()}')
print(f'Size of dataframe after removing duplicates: {train.shape}')

Sum of duplicated colums: 0
Size of dataframe before removing duplicates: (38053, 7)
Sum of duplicated colums after removing duplicates: 0
Size of dataframe after removing duplicates: (38053, 7)


#### Cross-validation Setup

In [44]:
# Step 1: Group by Image_ID and aggregate class labels into lists
train['new_class'] = train['class'].map(class_mapping)
grouped = train.groupby('Image_ID')['new_class'].apply(list).reset_index()

# Step 2: Create new columns based for each label
all_classes = train["class"].unique().tolist()
for unique_class in all_classes:
    grouped[unique_class] = 0


# Step 3: input 1 in that column if the label is in that image else 0
all_labels_list = (list(grouped['new_class'].values))
for train_index, label_List in enumerate(all_labels_list):
    unique_labels = list(set(label_List))
    for label_index in range(len(unique_labels)):
        label = int(unique_labels[label_index])
        for key_value in range(23):
            if label == key_value:
                grouped.loc[train_index, reverse_class_mapping[key_value]] = 1
                break

grouped

,Image_ID,new_class,Pepper_Bacterial_Spot,Pepper_Fusarium,Corn_Cercospora_Leaf_Spot,Corn_Common_Rust,Pepper_Leaf_Curl,Tomato_Early_Blight,Pepper_Cercospora,Tomato_Septoria,...,Corn_Healthy,Pepper_Healthy,Tomato_Healthy,Tomato_Late_Blight,Tomato_Fusarium,Pepper_Leaf_Blight,Tomato_Leaf_Curl,Tomato_Bacterial_Spot,Tomato_Mosaic,Corn_Northern_Leaf_Blight
0,id_11543h.jpg,"[0, 0, 0, 0]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_11ee1c.jpg,"[1, 1]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_11gglx.jpg,"[2, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3]",0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_11olmm.jpg,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_11siot.jpg,[4],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,id_zze2xq.jpg,"[1, 1, 11, 11, 11, 1]",0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4898,id_zzgx6e.jpg,"[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4899,id_zzm5ki.jpg,"[15, 15]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4900,id_zzr3ld.jpg,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Step 4: Apply `MultilabelStratifiedKFold` to make for a proper cross validation
X = grouped[['Image_ID']]
grouped['fold'] = -1
mskf = MultilabelStratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.random_state)
for i_fold, (train_index, test_index) in enumerate(mskf.split(X, grouped[all_classes])):
    grouped.loc[test_index, "fold"] = i_fold 
     
print(f"folds: {grouped['fold'].value_counts()}")
        

folds: fold
3    993
2    993
1    980
0    972
4    964
Name: count, dtype: int64


Update the Dataframe(grouped) with images_path

In [48]:
# create image_path for grouped_data
grouped['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in grouped.Image_ID]

# drop duplicates rows for test
test = test.drop_duplicates(subset=['Image_ID'], ignore_index=True)
test['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in test.Image_ID] 

BBOX Processing


In [ ]:
# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):

    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"

    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    # convert the dataframe to a dictionary
    dataframe = dataframe.to_dict('records')
    for i in tqdm(range(len(dataframe))):
        save_yolo_annotation(dataframe[i])



    # with multiprocessing.Pool(1) as pool:
    #     list(tqdm(pool.imap(save_yolo_annotation, dataframe.head().to_dict('records')), total=len(dataframe.head())))

Cross-Validation Implementation - Took about 149 minutes in running this code for 5 folds

In [ ]:
# create image_path for train data
train['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in train.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(class_mapping)

In [ ]:
#------------------------------- 

# for fold in range(CFG.folds):
for fold in range(0, 1): #runing for a single fold - first fold
    # images
    TRAIN_IMAGES_DIR = Path(ROOT_DIR / f'train/images/fold_{fold + 1}')
    VAL_IMAGES_DIR = Path(ROOT_DIR / f'val/images/fold_{fold + 1}')
    TEST_IMAGES_DIR = Path(ROOT_DIR / 'test/images')

    # labels
    TRAIN_LABELS_DIR = Path(ROOT_DIR / f'train/labels/fold_{fold + 1}')
    VAL_LABELS_DIR = Path(ROOT_DIR / f'val/labels/fold_{fold + 1}')
    TEST_LABELS_DIR = Path(ROOT_DIR / 'test/labels')


    DIRS = [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR]


    # get the train and val for that fold
    train_fold = grouped[grouped['fold'] != fold ].reset_index(drop=True)
    val_fold = grouped[grouped['fold'] == fold].reset_index(drop=True)

    # if code is running for the first time, then create a folder for test 
    if fold == 0:
        DIRS = [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_IMAGES_DIR, TEST_LABELS_DIR]
        

    # Create necessary directories
    for DIR in DIRS:
        if DIR.exists():
            shutil.rmtree(DIR)
        DIR.mkdir(parents=True, exist_ok=True)
       

    # Copy train, val, and test images to their respective dirs
    for img in tqdm(train_fold.image_path.unique()):
        shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])
    print(f'Copied train file for fold{fold+1} to folder')

    for img in tqdm(val_fold.image_path.unique()):
        shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])
    print(f'Copied val file for fold{fold+1} to folder')

    # For Test - if code is running for the first time, then move the images to the folder
    if fold == 0:
        for img in tqdm(test.image_path.unique()):
            shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])
        print(f'Copied test file for first fold to folder')


    X_train = train[train.Image_ID.isin(train_fold.Image_ID)].reset_index(drop=True)
    X_val = train[train.Image_ID.isin(val_fold.Image_ID)].reset_index(drop=True)


    # Save train and validation labels to their respective dirs
    process_dataset(X_train, TRAIN_LABELS_DIR)
    process_dataset(X_val, VAL_LABELS_DIR)


YAML File Creation

In [ ]:
# Create a data.yaml file required by YOLO
class_names = train['class'].unique().tolist()
num_classes = len(class_names)


# for fold in range(CFG.folds):
for fold in range(0, 1):
    # images
    TRAIN_IMAGES_DIR = Path(ROOT_DIR / f'train/images/fold_{fold + 1}')
    VAL_IMAGES_DIR = Path(ROOT_DIR / f'val/images/fold_{fold + 1}')


    data_yaml = {
        'train': str(TRAIN_IMAGES_DIR),
        'val': str(VAL_IMAGES_DIR),
        'nc': num_classes,
        'names': class_names
    }

    # Save the data.yaml file
    yaml_path = Path(ROOT_DIR / f'yaml/data_fold_{fold+1}.yaml')
    with open(yaml_path, 'w') as file:
        yaml.dump(data_yaml, file, default_flow_style=False)

Training and Evaluation

In [ ]:
# # Load a YOLO pretrained model

models = [
"rtdetr-l.pt",
"yolo11l.pt",
]

# for i in range(CFG.folds):
for i in range(0, 1):
    for model_name in models:
        print(f'Model name is {model_name}')
        print(f"-------------------Training for fold_{i}----------------")
        if 'yolo' in model_name:
            model = YOLO(model_name)
        else:
            model = RTDETR(model_name)
        
        model.train(
            data = f'yaml/data_fold_{i}.yaml',
            time=1,                                # Number of epochs - 1hour
            imgsz=640,               # Image size - 604image size
            batch=16,                  # Batch size - order of 32, 32, 16 
            device=CFG.device,               # Use the first GPU (0 for the first GPU)
            seed = CFG.seed,
            scale=0.7,
            degrees = 10,
            hsv_s = 0.2,
            hsv_v = 0.2,
            profile=True,
            erasing=0.2
        )



Inference on Validation Images For Fold-0

In [ ]:
# extract test images
VAL_IMAGES_DIR = ROOT_DIR / r'val\images\fold_0' # path to where val images for fold_0 are saved
image_files = os.listdir(VAL_IMAGES_DIR)
all_data = []

# create submission (subs) directory
os.makedirs(ROOT_DIR / 'subs', exist_ok=True)

MODEL_PATH = ROOT_DIR / 'runs\detect' # path to where all trained logs (images, weights) are saved

best_trained_models = []
for dirpath, dirnames, filenames in (os.walk(MODEL_PATH)):
    trained_models = [os.path.join(dirpath, file) for file in filenames if 'best.pt' in file]
    if trained_models:
        best_trained_models.extend(trained_models) # path to best_trained_models


for index, best_model_path in enumerate(best_trained_models):
    if index == 0:
        model = RTDETR(best_model_path)
    else:
        model = YOLO(best_model_path)
    for image_file in tqdm(image_files):
        img_path = os.path.join(VAL_IMAGES_DIR, image_file)
        results = model(img_path)

        boxes = results[0].boxes.xyxy.tolist()
        classes = results[0].boxes.cls.tolist()
        confidences = results[0].boxes.conf.tolist()
        names = results[0].names

        incorrect_prediction = 0

        if not boxes:
            incorrect_prediction = incorrect_prediction +   1
            all_data.append({
                'Image_ID': image_file,
                'class': 'NEG',
                'confidence': 1.0,
                'ymin': 0,
                'xmin': 0,
                'ymax': 0,
                'xmax': 0
            })
        else:
            for box, cls, conf in zip(boxes, classes, confidences):
                x1, y1, x2, y2 = box
                detected_class = names[int(cls)]

                all_data.append({
                    'Image_ID': image_file,
                    'class': detected_class,
                    'confidence': conf,
                    'ymin': y1,
                    'xmin': x1,
                    'ymax': y2,
                    'xmax': x2
                })
    print(f'========== Total Incorrect predction or dummy prediction {incorrect_prediction}================')
    # Convert the results to a DataFrame and save it
    sub = pd.DataFrame(all_data)
    # save to dir
    if index==0:
        sub.to_csv(ROOT_DIR / f'subs/RTDETR_submission.csv')
    else:
        sub.to_csv(ROOT_DIR / f'subs/YOLO_submission.csv')
